## import library

In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np


In [2]:
TRAIN_DIR = "images/train"
TEST_DIR = "images/test"

In [3]:
def Create_Data_Frame(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

## Training Data

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = Create_Data_Frame(TRAIN_DIR)

print(train['image'])

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed


surprise completed
0              images/train\angry\0.jpg
1              images/train\angry\1.jpg
2             images/train\angry\10.jpg
3          images/train\angry\10002.jpg
4          images/train\angry\10016.jpg
                      ...              
28816    images/train\surprise\9969.jpg
28817    images/train\surprise\9985.jpg
28818    images/train\surprise\9990.jpg
28819    images/train\surprise\9992.jpg
28820    images/train\surprise\9996.jpg
Name: image, Length: 28821, dtype: object


## Test Data

In [5]:
test = pd.DataFrame()
test['image'], test['label'] = Create_Data_Frame(TEST_DIR)

print(test)
print(test['image'])

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
706

In [6]:
def Extract_Features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [8]:
train_features = Extract_Features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [9]:
test_features = Extract_Features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [10]:
x_train = train_features/255.0
x_test = test_features/255.0

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [14]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [15]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [16]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=[48, 48, 1]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 [==============================] - 2553s 11s/step - loss: 1.8070 - accuracy: 0.2493 - val_loss: 1.7918 - val_accuracy: 0.2603
Epoch 2/100
226/226 [==============================] - 2257s 10s/step - loss: 1.7146 - accuracy: 0.2992 - val_loss: 1.5871 - val_accuracy: 0.3837
Epoch 3/100
226/226 [==============================] - 2375s 11s/step - loss: 1.5996 - accuracy: 0.3698 - val_loss: 1.5129 - val_accuracy: 0.4220
Epoch 4/100
226/226 [==============================] - 2885s 13s/step - loss: 1.5157 - accuracy: 0.4109 - val_loss: 1.4095 - val_accuracy: 0.4444
Epoch 5/100
226/226 [==============================] - 2647s 12s/step - loss: 1.4582 - accuracy: 0.4348 - val_loss: 1.3391 - val_accuracy: 0.4853
Epoch 6/100
226/226 [==============================] - 2354s 10s/step - loss: 1.4195 - accuracy: 0.4556 - val_loss: 1.3201 - val_accuracy: 0.5027
Epoch 7/100
226/226 [==============================] - 2439s 11s/step - loss: 1.3917 - accuracy: 0.4655 - val_loss: 1.2823 -

In [25]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)
model.save('emotiondetector.h5')

In [28]:
from keras.models import model_from_json
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [29]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'suprise']

In [30]:
def ef(image):
    img = load_img(image, grayscale = True)
    feature = np.array(img)
    feature.reshape((1, 48, 48, 1))
    return feature/255.0

In [40]:
image = 'images/train/angry/22.jpg'
print('original image is angry')
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)

original image is angry


ValueError: in user code:

    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline

image = 'images/train/sad/42.jpg'
print('original image is sad')
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)
plt.imshow(img.reshape(48, 48), cmap='gray')

original image is sad


c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


ValueError: in user code:

    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   